In [18]:
#importing libraries
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import bs4 as BeautifulSoup
import urllib.request  

#fetching the content from the URL
#fetched_data = urllib.request.urlopen('https://blog.floydhub.com/introduction-to-anomaly-detection-in-python.html')
#fetched_data = "Well, this should have been named \"Game of Starks\". Would have been more apt, they way the camera followed the Stark siblings only till the end!! Most disappointing end for a promising series i guess. I cant find all the adjectives to describe my angst. This series end could be the most boring, uninteresting, dry, sad, cruel, .... ever in TV history. Even mr. Martin says he is disappointed, but at least they provided a closure to series even if poor. Mr.Martin is never going to finish the books, as he got all his fingers into TV series pie. Easy and quick money! So, the fans have to make their good ending for the series in imadination and be content! I dont know why this series could not have a better happy ending even if spoils Mr.Martin's ego! As he is more into killing off characters. Screen writers to match or outdo mr.Martin morelike and messed it up!"
#article_read = fetched_data.read()

#parsing the URL content and storing in a variable
#article_parsed = BeautifulSoup.BeautifulSoup(article_read,'html.parser')

#returning <p> tags
#paragraphs = article_parsed.find_all('p')
#paragraphs = "Well, this should have been named \"Game of Starks\". Would have been more apt, they way the camera followed the Stark siblings only till the end!! Most disappointing end for a promising series i guess. I cant find all the adjectives to describe my angst. This series end could be the most boring, uninteresting, dry, sad, cruel, .... ever in TV history. Even mr. Martin says he is disappointed, but at least they provided a closure to series even if poor. Mr.Martin is never going to finish the books, as he got all his fingers into TV series pie. Easy and quick money! So, the fans have to make their good ending for the series in imadination and be content! I dont know why this series could not have a better happy ending even if spoils Mr.Martin's ego! As he is more into killing off characters. Screen writers to match or outdo mr.Martin morelike and messed it up!"
article_content = ''

#looping through the paragraphs and adding them to the variable
#for p in paragraphs:  
#    article_content += p.text

In [ ]:
def _create_dictionary_table(text_string) -> dict:
   
    #removing stop words
    stop_words = set(stopwords.words("english"))
    
    words = word_tokenize(text_string)
    
    #reducing words to their root form
    stem = PorterStemmer()
    
    #creating dictionary for the word frequency table
    frequency_table = dict()
    for wd in words:
        wd = stem.stem(wd)
        if wd in stop_words:
            continue
        if wd in frequency_table:
            frequency_table[wd] += 1
        else:
            frequency_table[wd] = 1

    return frequency_table

In [ ]:
def _calculate_sentence_scores(sentences, frequency_table) -> dict:   

    #algorithm for scoring a sentence by its words
    sentence_weight = dict()

    for sentence in sentences:
        sentence_wordcount = (len(word_tokenize(sentence)))
        sentence_wordcount_without_stop_words = 0
        for word_weight in frequency_table:
            if word_weight in sentence.lower():
                sentence_wordcount_without_stop_words += 1
                if sentence[:7] in sentence_weight:
                    sentence_weight[sentence[:7]] += frequency_table[word_weight]
                else:
                    sentence_weight[sentence[:7]] = frequency_table[word_weight]

        sentence_weight[sentence[:7]] = sentence_weight[sentence[:7]] / sentence_wordcount_without_stop_words

       

    return sentence_weight

In [ ]:
def _calculate_average_score(sentence_weight) -> int:
   
    #calculating the average score for the sentences
    sum_values = 0
    for entry in sentence_weight:
        sum_values += sentence_weight[entry]

    #getting sentence average value from source text
    average_score = (sum_values / len(sentence_weight))

    return average_score

In [ ]:
def _get_article_summary(sentences, sentence_weight, threshold):
    sentence_counter = 0
    article_summary = ''

    for sentence in sentences:
        if sentence[:7] in sentence_weight and sentence_weight[sentence[:7]] >= (threshold):
            article_summary += " " + sentence
            sentence_counter += 1

    return article_summary

In [ ]:
def _run_article_summary(article):
    
    #creating a dictionary for the word frequency table
    frequency_table = _create_dictionary_table(article)

    #tokenizing the sentences
    sentences = sent_tokenize(article)

    #algorithm for scoring a sentence by its words
    sentence_scores = _calculate_sentence_scores(sentences, frequency_table)

    #getting the threshold
    threshold = _calculate_average_score(sentence_scores)

    #producing the summary
    article_summary = _get_article_summary(sentences, sentence_scores, 1.5 * threshold)

    return article_summary

In [ ]:
if __name__ == '__main__':
    summary_results = _run_article_summary(article_content)
    print(summary_results)